In [37]:
import os
file_dir = r".\llm-20-questions"
discuss_path = file_dir + r"\discuss.json"
kernels_path = file_dir + r"\kernels.json"
output_dir = file_dir + "_jp"

In [38]:
#jsonファイルを読み込む
import json

with open(discuss_path, "r", encoding="utf-8") as f:
    discuss_info = json.load(f)

with open(kernels_path, "r", encoding="utf-8") as f:
    kernels_info = json.load(f)

In [39]:
#discuss_infoを"votes"順に並び替え
discuss_info_sort_by_vite = sorted(discuss_info, key=lambda x: x["votes"], reverse=True)

#discuss_infoを"comments"順に並び替え
discuss_info_sort_by_comments = sorted(discuss_info, key=lambda x: x["comments"], reverse=True)

#kernel_infoを"totalVotes  "順に並び替え
kernels_info_sort_by_totalVotes = sorted(kernels_info, key=lambda x: x["totalVotes  "], reverse=True)

In [40]:
import re
#リンク一覧
discuss_link_sort_by_vite = []
discuss_link_sort_by_comments = []
kernels_directory_sort_by_totalVotes = []

for discuss in discuss_info_sort_by_vite:
  file_name = discuss['title']
  # エスケープシーケンスを削除
  file_name = re.sub(r'\\x[0-9a-fA-F]{2}', '', file_name)

  # 無効な文字を置換または除去
  file_name = re.sub(r'[\.=<>:"/\\|?*]', '', file_name)
  file_name = file_name.replace(" ", "_")
  if len(file_name) > 60:
      file_name = file_name[:59]+"…"
  file_name = file_name + ".md"
  link = "./discuss/" + file_name
  discuss_link_sort_by_vite.append(link)

for discuss in discuss_info_sort_by_comments:
  file_name = discuss['title']
  # エスケープシーケンスを削除
  file_name = re.sub(r'\\x[0-9a-fA-F]{2}', '', file_name)

  # 無効な文字を置換または除去
  file_name = re.sub(r'[\.=<>:"/\\|?*]', '', file_name)
  file_name = file_name.replace(" ", "_")
  if len(file_name) > 60:
      file_name = file_name[:59]+"…"
  file_name = file_name + ".md"
  link = "./discuss/" + file_name
  discuss_link_sort_by_comments.append(link)

for kernel in kernels_info_sort_by_totalVotes:
  directory = kernel['title']
  # エスケープシーケンスを削除
  directory = re.sub(r'\\x[0-9a-fA-F]{2}', '', directory)

  # 無効なファイル名の文字を定義
  invalid_chars = r'<>:"/\|?*'

  # 無効な文字を置換または除去
  directory = re.sub(r'[\.=<>:"/\\|?*]', '', directory)
  directory = directory.replace(" ", "_")
  if len(directory) > 60:
      directory = directory[:59]+"…"

  kernels_directory_sort_by_totalVotes.append(directory)

In [41]:
#各kernels_directory_sort_by_totalVotesの中に一つipynbファイルがあるので、そのファイル名を取得

import os
kernels_ipynb_sort_by_totalVotes = []
for directory in kernels_directory_sort_by_totalVotes:
  for file in os.listdir(file_dir + r"\kernels" + "\\" + directory):
    if file.endswith(".ipynb"):
      kernels_ipynb_sort_by_totalVotes.append(file)
      break
#kernels_directory_sort_by_totalVotesとkernels_ipynb_sort_by_totalVotesをzipして、リンクを作成
kernels_link_sort_by_totalVotes = []
for directory, ipynb in zip(kernels_directory_sort_by_totalVotes, kernels_ipynb_sort_by_totalVotes):
  link = "./kernels/" + directory + "/" + ipynb
  kernels_link_sort_by_totalVotes.append(link)

In [42]:
#全てのディスカッションを結合した一つのマークダウンファイルを作成
mdfile_discuss_all = ""
mdfile_discuss_all_jp = ""
for i,link in enumerate(discuss_link_sort_by_vite):
  mdfile_discuss_all += f"* --- discussion numver {i}, the number of votes :{discuss_info_sort_by_vite[i]['votes']} ---\n\n"
  mdfile_discuss_all_jp += f"* --- discussion numver {i}, the number of votes :{discuss_info_sort_by_vite[i]['votes']} ---\n\n"
  #ファイルを読み込む
  with open(file_dir + discuss_link_sort_by_vite[i][1:], "r", encoding="utf-8") as f:
    mdfile_discuss_all += f.read()
  with open(output_dir + discuss_link_sort_by_vite[i][1:], "r", encoding="utf-8") as f:
    mdfile_discuss_all_jp += f.read()
  mdfile_discuss_all += "\n\n"
  mdfile_discuss_all_jp += "\n\n"

In [43]:
#ファイルを書き込む
with open(file_dir + r"\discuss_all.md", "w", encoding="utf-8") as f:
  f.write(mdfile_discuss_all)

with open(output_dir + r"\discuss_all.md", "w", encoding="utf-8") as f:
    f.write(mdfile_discuss_all_jp)

In [44]:
len(mdfile_discuss_all), len(mdfile_discuss_all_jp)

(383215, 241436)

In [45]:
import nbformat

#全てのディスカッションを結合した一つのマークダウンファイルを作成
mdfile_kernels_all = ""
mdfile_kernels_all_jp = ""
for i,link in enumerate(kernels_link_sort_by_totalVotes):
  mdfile_kernels_all += f"** @@@ Jupyter Notebook numver {i}, the number of votes :{kernels_info_sort_by_totalVotes[i]['totalVotes  ']} @@@ **\n\n"
  mdfile_kernels_all_jp += f"** @@@ Jupyter Notebook numver {i}, the number of votes :{kernels_info_sort_by_totalVotes[i]['totalVotes  ']} @@@ **\n\n"
  #ファイルを読み込む
  with open(file_dir + kernels_link_sort_by_totalVotes[i][1:], "r", encoding="utf-8") as f:
    notebook = nbformat.read(f, as_version=4)
    cells = [cell for cell in notebook.cells if cell.cell_type == "markdown" or cell.cell_type == "code"]
    for j,cell in enumerate(cells):
      if cell.cell_type == "markdown":
        mdfile_kernels_all += f"---The following area is a Markdown cell (cell numver is {j})---\n```markdown\n"
        mdfile_kernels_all += cell.source.strip() + "\n```\n\n"
      elif cell.cell_type == "code":
        mdfile_kernels_all += f"---The following area is a Code cell (cell numver is {j})---\n```python\n"
        mdfile_kernels_all += cell.source + "\n```\n\n"
  with open(output_dir + kernels_link_sort_by_totalVotes[i][1:], "r", encoding="utf-8") as f:
    notebook = nbformat.read(f, as_version=4)
    cells = [cell for cell in notebook.cells if cell.cell_type == "markdown" or cell.cell_type == "code"]
    #cellsのindex番号1を削除
    cells.pop(1)
    cells[0].source = cells[0].source.strip()
    if cells[0].source.endswith('---'):
      cells[0].source = cells[0].source[:-3]
    for j,cell in enumerate(cells):
      if cell.cell_type == "markdown":
        mdfile_kernels_all_jp += f"---The following area is a Markdown cell (cell numver is {j})---\n```markdown\n"
        mdfile_kernels_all_jp += cell.source.strip() + "\n```\n\n"
      elif cell.cell_type == "code":
        mdfile_kernels_all_jp += f"---The following area is a Code cell (cell numver is {j})---\n```python\n"
        mdfile_kernels_all_jp += cell.source + "\n```\n\n"

In [46]:
#ファイルを書き込む
with open(file_dir + r"\kernels_all.md", "w", encoding="utf-8") as f:
  f.write(mdfile_kernels_all)

with open(output_dir + r"\kernels_all.md", "w", encoding="utf-8") as f:
  f.write(mdfile_kernels_all_jp)

In [47]:
print(len(mdfile_kernels_all), len(mdfile_kernels_all_jp))

853912 851886


In [48]:
#全てのディスカッションと全てのJupyter Notebookを結合した一つのマークダウンファイルを作成
mdfile_all_jp = "# 全てのディスカッション\n\n" + mdfile_discuss_all_jp + "\n\n# 全てのJupyter Notebook\n\n" + mdfile_kernels_all_jp

mdfile_all = "# All Discussions\n\n" + mdfile_discuss_all + "\n\n# All Jupyter Notebooks\n\n" + mdfile_kernels_all

#ファイルを書き込む
with open(file_dir + r"\all.md", "w", encoding="utf-8") as f:
  f.write(mdfile_all)

with open(output_dir + r"\all.md", "w", encoding="utf-8") as f:
  f.write(mdfile_all_jp)